# Interface for Pipeline Execution
## (EUR-USD exchange data)

In [18]:
# Import models, metrics, paths and functions
from utils.helpers import csv_reader
from utils.paths import *
from pipeline.run_pipeline import run_pipeline

from models import MODELS
from metrics import METRICS

In [19]:
# Get current models and metrics
print(METRICS)
print(MODELS)

{'MAE': <function mean_absolute_error at 0x000002D98E3F5620>, 'RMSE': <function root_mean_squared_error at 0x000002D98E520220>, 'MAPE': <function mean_absolute_percentage_error at 0x000002D98E3F59E0>, 'sMAPE': <function symmetric_mean_absolute_percentage_error at 0x000002D98E520360>}
{'FORECASTERS': {'without_covariates': {'Naive': {'model': <class 'sktime.forecasting.naive.NaiveForecaster'>, 'package': 'sktime', 'options': {'strategy': 'last'}}, 'Naive (drift)': {'model': <class 'sktime.forecasting.naive.NaiveForecaster'>, 'package': 'sktime', 'options': {'strategy': 'drift'}}, 'AutoSARIMA': {'model': <class 'sktime.forecasting.arima.AutoARIMA'>, 'package': 'sktime', 'options': {'seasonal': True, 'stationary': False, 'd': 1, 'trace': False, 'update_pdq': False, 'with_intercept': 'auto', 'max_p': 6, 'max_q': 6, 'suppress_warnings': True}}, 'Exponential Smoothing': {'model': <class 'sktime.forecasting.exp_smoothing.ExponentialSmoothing'>, 'package': 'sktime', 'options': {}}, 'STL': {'mo

In [20]:
## Using the EUR-USD Exchange Rate data

# Read EUR-USD exchange data
df = csv_reader(TESTDATA_DIR, 'eurusd', columns=['datetime', 'bid_close'])

# Use a small subset of the original data for testing:
df = df.iloc[:10000, :]
display(df)

,bid_close
datetime,
2020-01-01 00:00:00,NaN
2020-01-01 00:15:00,NaN
2020-01-01 00:30:00,NaN
2020-01-01 00:45:00,NaN
2020-01-01 01:00:00,NaN
...,...
2020-04-14 02:45:00,1.09
2020-04-14 03:00:00,1.09
2020-04-14 03:15:00,1.09


In [31]:
output_dict = (
    run_pipeline(
        df=df, models=MODELS, metrics=METRICS,
        agg_method='last', agg_freq='B',
        forecast_init_train=0.3, ensemble_init_train=0.3,
        #select_forecasters=['Naive', 'Naive (drift)', 'ExponentialSmoothing', 'STL', 'XGBoost'],
        select_forecaster = 'all',
        select_ensemblers='all',
        verbose=True
    )
)

[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...
[2024-03-06 19:01] Starting  Pipeline...

== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==


== Starting Step 1 in Pipeline: Data Preprocessing ==

Searching for time information...
Searching for time information...
Searching for time information...
Searching for time inform

Dates found in 'index' column!
Dates found in 'index' column!
Dates found in 'index' column!
Dates found in 'index' column!
Dates found in 'index' column!
Dates found in 'index' column!
Dates found in 'index' column!
Dates found in 'index' column!
Dates found in 'index' column!
Inferred frequency: 15T
Inferred frequency: 15T
Inferred frequency: 15T
Inferred frequency: 15T
Inferred frequency: 15T
Inferred frequency: 15T
Inferred frequency: 15T
Inferred frequency: 15T
Inferred frequency: 15T
Data from goes from 2020-01-01 to 2020-04-14, resulting in 10000 observations.

Data from goes from 2020-01-01 to 2020-04-14, resulting in 10000 observations.

Data from goes from 2020-01-01 to 2020-04-14, resulting in 10000 observations.

Data from goes from 2020-01-01 to 2020-04-14, resulting in 10000 observations.

Data from goes from 2020-01-01 to 2020-04-14, resulting in 10000 observations.

Data from goes from 2020-01-01 to 2020-04-14, resulting in 10000 observations.

Data from goes from 2020-

In [35]:
import pandas as pd

pd.set_option('display.float_format', '{:.2f}'.format)
# Format performance metric values
def format_numeric(val):
    if isinstance(val, float):
        return '{:.6f}'.format(val)
    return val
formatted_metrics = output_dict['metrics ranking'].applymap(format_numeric)

display(formatted_metrics.style.hide())


Model,MAE,RMSE,MAPE,sMAPE,MAE Ranking,RMSE Ranking,MAPE Ranking,sMAPE Ranking
Weighted Ensemble: Inverse Error Covariance,0.006268,0.008330,0.005692,0.379191,1,1,1,1
Naive,0.006643,0.008753,0.006043,0.402667,2,2,2,2
Exponential Smoothing,0.006655,0.008763,0.006054,0.403417,3,3,3,3
Meta Ensemble: Linear Regression,0.006743,0.008912,0.006138,0.408527,4,5,4,4
Naive (drift),0.006826,0.008873,0.006211,0.413911,5,4,5,5
XGBoost,0.007547,0.009868,0.006833,0.456078,6,6,6,6
Weighted Ensemble: Inverse RMSE,0.007669,0.009893,0.006972,0.464799,7,7,7,7
Weighted Ensemble: Inverse Variance,0.007773,0.009915,0.007063,0.470917,8,8,8,8
Weighted Ensemble: Simple,0.008050,0.010284,0.007317,0.487915,9,10,9,9
Meta Ensemble: RandomForest,0.008212,0.010019,0.007454,0.496681,10,9,10,10
